# #6 Streak Leaderboard

In [1]:
import polars as pl

In [2]:
active_date = pl.date(year=2025, month=9, day=28)
lesson_completion = (
    pl.scan_csv("LessonStreaks.csv", try_parse_dates=True)
    .unique(["user_id", "date"]) # drop instance of users completing multiple lessons on the same day
    .sort(["user_id", "date"])   # sort sequence of events by user ID and date
    .with_columns(
        streak = 
        pl.col("date")
        .diff()                  # day-to-day gaps (Detect breaks in consecutive days)
        .dt.total_days()
        .ne(1)                   # ne(1) → True when the streak breaks
        .fill_null(True)
        .cast(pl.Int64)
        .cum_sum()
        .over("user_id")         # cum_sum().over("user_id") → unique streak id per user
    )
)

lesson_completion.head().collect()

id,lesson_id,date,user_id,user_name,streak
i64,i64,date,i64,str,i64
7839783,63658156,2025-05-01,8226413,"""Aeris Stone""",1
7872349,63658157,2025-05-05,8226413,"""Aeris Stone""",2
7876346,63658171,2025-05-06,8226413,"""Aeris Stone""",2
7882741,63658176,2025-05-07,8226413,"""Aeris Stone""",2
7956420,64983797,2025-05-21,8226413,"""Aeris Stone""",3


In [3]:
streaks = (
    lesson_completion
    .group_by(["user_id", "user_name", "streak"])
    .agg(
        streak_len = pl.len(),
        start = pl.col("date").min(),
        end = pl.col("date").max()
    )
)

streaks.sort("streak_len", descending=True).head().collect()

user_id,user_name,streak,streak_len,start,end
i64,str,i64,u32,date,date
84071240,"""Stan Pancake""",1,151,2025-05-01,2025-09-28
16814590,"""Tom Parmesan""",3,121,2025-05-31,2025-09-28
215518363,"""Chuck Butternut""",2,120,2025-06-01,2025-09-28
215517194,"""Maverick Banner""",2,118,2025-06-01,2025-09-26
215468074,"""Strange Espresso""",2,118,2025-06-01,2025-09-26


In [4]:
leaderboard = (
    streaks
    .filter(pl.col("end") == active_date)
    .sort("streak_len", descending=True)
    .limit(10)
    .collect()
)

leaderboard

user_id,user_name,streak,streak_len,start,end
i64,str,i64,u32,date,date
84071240,"""Stan Pancake""",1,151,2025-05-01,2025-09-28
16814590,"""Tom Parmesan""",3,121,2025-05-31,2025-09-28
215518363,"""Chuck Butternut""",2,120,2025-06-01,2025-09-28
213142099,"""Helen Melon""",6,96,2025-06-25,2025-09-28
215121049,"""Gertrude Strudel""",3,88,2025-07-03,2025-09-28
176891675,"""Scott Pockett""",3,78,2025-07-13,2025-09-28
227438241,"""Demetrius Trumpet""",1,76,2025-07-15,2025-09-28
199107800,"""Shrimp Pringle""",15,53,2025-08-07,2025-09-28
210092431,"""Skeeter Weaselwood""",5,51,2025-08-09,2025-09-28
